Features:

- Days

- Rainfall indicates the quantity of rain falling (mm)

- Temperature indicates the temperature (°C)

- Volume indicates the volume of water taken from the drinking water treatment plant (m**3)

- Hydrometry indicates the groundwater level (m)

Target:

- Depth to Groundwater indicates the groundwater level (m from the ground floor)



---

\\

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np 
from sklearn.preprocessing import RobustScaler

Days (0-365)

In [ ]:
days = []
for i in range(1,366,1):
  days.append(i)
days

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
 185

Depth Ground Water (30-32m Max limit of changes in depth)

In [ ]:
from random import uniform

x = 1
depth = []
while x!=366:
    depth.append(round(uniform(30, 32), 2)) # water level randomised between 30 & 40 m
    x+=1

Rain (India average: 400 - 500 mm) 

In [ ]:
x = 1
rain = []
while x!=366:
    rain.append(round(uniform(400, 500), 2)) 
    x+=1

Temperature (15 - 25 degree celsius)

In [ ]:
x = 1
temp = []
while x!=366:
    temp.append(round(uniform(16, 25), 2)) 
    x+=1

Hydrometry / Groundwater Level (15-20) ***(Target)***

In [ ]:
x = 1
level = []
while x!=366:
    level.append(round(uniform(15, 20), 2)) 
    x+=1

Joining Columns

In [ ]:
arr = [days, depth, rain, temp, level] # put all columns in array and convert to dataframe.
arr = pd.DataFrame(arr) # convert
arr = arr.T # Transposing/Rotating by 90 degree
arr.columns = ['days', 'depth', 'rain', 'temp', 'level']
arr = arr.drop(['days'], axis = 1) # Days not reqd. Index is sufficient
arr

,depth,rain,temp,level
0,31.82,460.93,23.82,19.22
1,31.77,441.92,18.11,15.38
2,30.09,406.02,21.77,19.08
3,31.90,406.05,23.67,15.50
4,31.83,442.17,21.01,17.24
...,...,...,...,...
360,31.03,489.76,21.15,15.50
361,31.75,408.42,24.60,16.26
362,31.22,411.32,22.04,15.33
363,31.25,472.28,21.08,18.65


Features Column

In [ ]:
X = arr.iloc[:,0:-1] # [row1:row2, column1:column2]
X

,depth,rain,temp
0,31.82,460.93,23.82
1,31.77,441.92,18.11
2,30.09,406.02,21.77
3,31.90,406.05,23.67
4,31.83,442.17,21.01
...,...,...,...
360,31.03,489.76,21.15
361,31.75,408.42,24.60
362,31.22,411.32,22.04
363,31.25,472.28,21.08


Target Column

In [ ]:
Y = arr['level']
Y

0      19.22
1      15.38
2      19.08
3      15.50
4      17.24
       ...  
360    15.50
361    16.26
362    15.33
363    18.65
364    16.58
Name: level, Length: 365, dtype: float64

In [ ]:
arr.to_csv('data.csv')